In [1]:
#setup - rememeber to switch to tensorflow 2.3 kernel...
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import scipy.io as sio
import datetime

#need to have these two lines to work on my ancient 1060 3gb
#  https://stackoverflow.com/questions/43990046/tensorflow-blas-gemm-launch-failed
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)
print(tf.__version__)

%matplotlib notebook
%load_ext tensorboard
%load_ext autoreload
%autoreload 2
%autosave 180

C:\Users\Derm\anaconda3\envs\tf23\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Derm\anaconda3\envs\tf23\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\Derm\anaconda3\envs\tf23\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  stacklevel=1)


2.3.0


Autosaving every 180 seconds


In [2]:
#load individual data numpy files
#-------------------------------------------------------------------
# d1_1 = np.load('C:/Users/Derm/Desktop/big/pshift/test1_scan1.npy')
# d2_1 = np.load('C:/Users/Derm/Desktop/big/pshift/test1_scan2.npy')
# y1 = np.load('C:/Users/Derm/Desktop/big/pshift/test1_ground_truth.npy')

# d1_2 = np.load('C:/Users/Derm/Desktop/big/pshift/test2_scan1.npy')
# d2_2 = np.load('C:/Users/Derm/Desktop/big/pshift/test2_scan2.npy')
# y2 = np.load('C:/Users/Derm/Desktop/big/pshift/test2_ground_truth.npy')

d1_1 = np.load("C:/Users/Derm/Desktop/big/pshift/ICET_Ford_v1_scan1.npy")
d2_1 = np.load("C:/Users/Derm/Desktop/big/pshift/ICET_Ford_v1_scan2.npy")
gt_1 = np.load("C:/Users/Derm/Desktop/big/pshift/ICET_Ford_v1_ground_truth.npy")
# d1_1 = np.load("C:/Users/Derm/Desktop/big/pshift/ICET_Ford_v3_scan1.npy")
# d2_1 = np.load("C:/Users/Derm/Desktop/big/pshift/ICET_Ford_v3_scan2.npy")
# gt_1 = np.load("C:/Users/Derm/Desktop/big/pshift/ICET_Ford_v3_ground_truth.npy")

d1_2 = np.load("C:/Users/Derm/Desktop/big/pshift/ICET_Ford_v2_scan1.npy")
d2_2 = np.load("C:/Users/Derm/Desktop/big/pshift/ICET_Ford_v2_scan2.npy")
gt_2 = np.load("C:/Users/Derm/Desktop/big/pshift/ICET_Ford_v2_ground_truth.npy")
# d1_2 = np.loadtxt("C:/Users/Derm/Desktop/big/pshift/ICET_Ford_v2_scan1.txt")
# d2_2 = np.loadtxt("C:/Users/Derm/Desktop/big/pshift/ICET_Ford_v2_scan2.txt")
# gt_2 = np.loadtxt("C:/Users/Derm/Desktop/big/pshift/ICET_Ford_v2_ground_truth.txt")

d1 = np.append(d1_1, d1_2, axis = 0)
d2 = np.append(d2_1, d2_2, axis = 0)
gt = np.append(gt_1, gt_2, axis = 0)

# #TODO: loop through all files in testing folder 
# #combine scans 1 and 2 from both files
#-------------------------------------------------------------------
#single big file
# d1 = np.load("C:/Users/Derm/Desktop/big/pshift/scan1_300k_50_samples.npy")
# d2 = np.load("C:/Users/Derm/Desktop/big/pshift/scan2_300k_50_samples.npy")
# gt = np.load("C:/Users/Derm/Desktop/big/pshift/ground_truth_300k_50_samples.npy")
#-------------------------------------------------------------------

#reshape but don't convert to tensor
points_per_sample = 50          #poitns sammpled from each voxel
tsplit = 0.95                   #this fraction goes into training

scan1 = np.reshape(d1, [-1, points_per_sample, 3])
scan2 = np.reshape(d2, [-1, points_per_sample, 3])
ntrain = int(tsplit*tf.shape(scan1)[0].numpy())

x_train = np.append(scan1[:ntrain], scan2[:ntrain], axis = 1)
x_test = np.append(scan1[ntrain:], scan2[ntrain:], axis = 1)
# print(np.shape(x_train))
# print(np.shape(x_test))

y_train = gt[:ntrain]
y_test = gt[ntrain:]
# print(np.shape(y_train))
# print(np.shape(y_test))

#-------------------------------------------------------------------



In [3]:
#train network
from network import Net

runLen = 300

def scheduler(epoch, learning_rate):
    part1 = runLen//3
    part2 = 2*runLen//3
    if epoch < part1:
        learning_rate = 0.002
        return learning_rate
    if epoch >= part1 and epoch < part2:
        learning_rate = 0.001
        return learning_rate
    if epoch >= part2:
        learning_rate = 0.00025
        return learning_rate

model = Net()
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.002),
              loss = tf.keras.losses.MeanAbsoluteError()) #was MeanSquaredError()

summary = model.summary()
print(summary)

scheduler = tf.keras.callbacks.LearningRateScheduler(scheduler)
cp = tf.keras.callbacks.ModelCheckpoint("FordNetCP.kmod", monitor = 'val_loss', save_best_only = True) 

log_dir = "runs/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

trace = model.fit(x = x_train, y = y_train, batch_size = 512, epochs=runLen, verbose=1, 
                  validation_split = 0.2, shuffle=True, callbacks = [cp]) #scheduler

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 100, 3)]          0         
_________________________________________________________________
tf_op_layer_ExpandDims (Tens [(None, 100, 3, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 100, 1, 64)        256       
_________________________________________________________________
batch_normalization (BatchNo (None, 100, 1, 64)        256       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 100, 1, 64)        4160      
_________________________________________________________________
batch_normalization_1 (Batch (None, 100, 1, 64)        256       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 100, 1, 64)       

INFO:tensorflow:Assets written to: FordNetCP.kmod\assets
906/906 [==============================] - 21s 24ms/step - loss: 0.1525 - val_loss: 0.1192
Epoch 3/300
904/906 [============================>.] - ETA: 0s - loss: 0.1354WARNING:tensorflow:AutoGraph could not transform <function canonicalize_signatures.<locals>.signature_wrapper at 0x000001FAF9EB9708> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function with @tf.autograph.experimental.

INFO:tensorflow:Assets written to: FordNetCP.kmod\assets
906/906 [==============================] - 21s 24ms/step - loss: 0.0588 - val_loss: 0.0406
Epoch 40/300
906/906 [==============================] - 19s 21ms/step - loss: 0.0585 - val_loss: 0.0413
Epoch 41/300
906/906 [==============================] - 20s 22ms/step - loss: 0.0581 - val_loss: 0.0482
Epoch 42/300
906/906 [==============================] - 19s 21ms/step - loss: 0.0574 - val_loss: 0.0461
Epoch 43/300
906/906 [==============================] - 19s 21ms/step - loss: 0.0607 - val_loss: 0.0488
Epoch 44/300
906/906 [==============================] - 19s 21ms/step - loss: 0.0608 - val_loss: 0.0499
Epoch 45/300
906/906 [==============================] - 19s 21ms/step - loss: 0.0576 - val_loss: 0.0491
Epoch 46/300
906/906 [==============================] - 19s 21ms/step - loss: 0.0579 - val_loss: 0.0417
Epoch 47/300
906/906 [==============================] - 19s 21ms/step - loss: 0.0571 - val_loss: 0.0434
Epoch 48/300
906/906

906/906 [==============================] - 19s 21ms/step - loss: 0.0460 - val_loss: 0.0415
Epoch 125/300
906/906 [==============================] - 19s 21ms/step - loss: 0.0462 - val_loss: 0.0412
Epoch 126/300
906/906 [==============================] - 19s 21ms/step - loss: 0.0459 - val_loss: 0.0381
Epoch 127/300
906/906 [==============================] - 19s 21ms/step - loss: 0.0462 - val_loss: 0.0348
Epoch 128/300
906/906 [==============================] - 19s 21ms/step - loss: 0.0463 - val_loss: 0.0369
Epoch 129/300
906/906 [==============================] - 19s 21ms/step - loss: 0.0463 - val_loss: 0.0374
Epoch 130/300
906/906 [==============================] - 19s 21ms/step - loss: 0.0466 - val_loss: 0.0425
Epoch 131/300
906/906 [==============================] - 19s 21ms/step - loss: 0.0463 - val_loss: 0.0363
Epoch 132/300
906/906 [==============================] - 19s 21ms/step - loss: 0.0462 - val_loss: 0.0405
Epoch 133/300
906/906 [==============================] - 20s 22ms/ste

906/906 [==============================] - 19s 21ms/step - loss: 0.0434 - val_loss: 0.0358
Epoch 230/300
906/906 [==============================] - 19s 21ms/step - loss: 0.0429 - val_loss: 0.0358
Epoch 231/300
906/906 [==============================] - 19s 21ms/step - loss: 0.0427 - val_loss: 0.0411
Epoch 232/300
906/906 [==============================] - 19s 21ms/step - loss: 0.0431 - val_loss: 0.0358
Epoch 233/300
906/906 [==============================] - 19s 21ms/step - loss: 0.0428 - val_loss: 0.0368
Epoch 234/300
906/906 [==============================] - 19s 21ms/step - loss: 0.0423 - val_loss: 0.0330
Epoch 235/300
906/906 [==============================] - 19s 21ms/step - loss: 0.0426 - val_loss: 0.0330
Epoch 236/300
906/906 [==============================] - 19s 21ms/step - loss: 0.0424 - val_loss: 0.0360
Epoch 237/300
906/906 [==============================] - 19s 21ms/step - loss: 0.0421 - val_loss: 0.0329
Epoch 238/300
906/906 [==============================] - 19s 21ms/ste

906/906 [==============================] - 20s 22ms/step - loss: 0.0416 - val_loss: 0.0375
Epoch 298/300
906/906 [==============================] - 19s 21ms/step - loss: 0.0409 - val_loss: 0.0381
Epoch 299/300
906/906 [==============================] - 19s 21ms/step - loss: 0.0412 - val_loss: 0.0415
Epoch 300/300
906/906 [==============================] - 19s 21ms/step - loss: 0.0411 - val_loss: 0.0348


In [ ]:
#save stuff
# np.save("C:/Users/Derm/Desktop/big/pshift/ICET_Ford_v2_scan1", d1_2)
# np.save("C:/Users/Derm/Desktop/big/pshift/ICET_Ford_v2_scan2", d2_2)
# np.save("C:/Users/Derm/Desktop/big/pshift/ICET_Ford_v2_ground_truth", gt_2)

# d1_2 = np.loadtxt("C:/Users/Derm/Desktop/big/pshift/ICET_Ford_v1_scan1.txt")
# d2_2 = np.loadtxt("C:/Users/Derm/Desktop/big/pshift/ICET_Ford_v1_scan2.txt")
# gt_2 = np.loadtxt("C:/Users/Derm/Desktop/big/pshift/ICET_Ford_v1_ground_truth.txt")

In [ ]:
print(np.shape(x_train))